In [1]:
from __future__ import print_function, division
import requests
import pandas as pd
import numpy as np
import os.path
from bs4 import BeautifulSoup

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")

%matplotlib inline

from sklearn.linear_model import LinearRegression, Lasso, LassoCV
import statsmodels.formula.api as smf

In [2]:

import requests

dict1 = {}
seasons = list(range(2021, 2011, -1))
for year in seasons:
    dict1[year] = 'https://www.basketball-reference.com/leagues/NBA_{}_per_game.html'.format(year)

In [4]:
url2021 = 'https://www.basketball-reference.com/leagues/NBA_2021_per_game.html'
url2020 = 'https://www.basketball-reference.com/leagues/NBA_2020_per_game.html'
url2019 = 'https://www.basketball-reference.com/leagues/NBA_2019_per_game.html'
url2018 = 'https://www.basketball-reference.com/leagues/NBA_2018_per_game.html'
url2017 = 'https://www.basketball-reference.com/leagues/NBA_2017_per_game.html'
url2016 = 'https://www.basketball-reference.com/leagues/NBA_2016_per_game.html'
url2015 = 'https://www.basketball-reference.com/leagues/NBA_2015_per_game.html'
url2014 = 'https://www.basketball-reference.com/leagues/NBA_2014_per_game.html'
url2013 = 'https://www.basketball-reference.com/leagues/NBA_2013_per_game.html'
url2012 = 'https://www.basketball-reference.com/leagues/NBA_2012_per_game.html'

In [5]:
response = requests.get(url2021)
page = response.text
soup = BeautifulSoup(page, "lxml")

#Assigning header row to column
headers = [th.getText() for th in soup.findAll('tr', limit=2)[0].findAll('th')]
headers = headers[1:]
#Web scrape text from table rows excluding header
rows = soup.find_all('tr')[1:]
player_stats = [[td.getText() for td in rows[i].find_all('td')]
            for i in range(len(rows))]
#Adding 2021 to df
statstotal = pd.DataFrame(player_stats, columns = headers)
statstotal['YEAR'] = 2021


In [8]:
url_list = [url2020, url2019, url2018, url2017, url2016, url2015, url2014, url2013, url2012]

In [11]:
for index, i in enumerate(url_list):
    response = requests.get(i)
    page = response.text
    soup = BeautifulSoup(page, "lxml")
    rows = soup.find_all('tr')[1:]
    player_stats2 = [[td.getText() for td in rows[r].find_all('td')] for r in range(len(rows))]
    stats = pd.DataFrame(player_stats2, columns = headers)
    stats['YEAR'] = 2021-(index+1)
    statstotal = statstotal.append(stats, ignore_index=True)

                       Player Pos Age   Tm   G  GS    MP   FG   FGA   FG%  \
0                Steven Adams   C  26  OKC  63  63  26.7  4.5   7.6  .592   
1                 Bam Adebayo  PF  22  MIA  72  72  33.6  6.1  11.0  .557   
2           LaMarcus Aldridge   C  34  SAS  53  53  33.1  7.4  15.0  .493   
3              Kyle Alexander   C  23  MIA   2   0   6.5  0.5   1.0  .500   
4    Nickeil Alexander-Walker  SG  21  NOP  47   1  12.6  2.1   5.7  .368   
..                        ...  ..  ..  ...  ..  ..   ...  ...   ...   ...   
672                Trae Young  PG  21  ATL  60  60  35.3  9.1  20.8  .437   
673               Cody Zeller   C  27  CHO  58  39  23.1  4.3   8.3  .524   
674              Tyler Zeller   C  30  SAS   2   0   2.0  0.5   2.0  .250   
675                Ante Žižić   C  23  CLE  22   0  10.0  1.9   3.3  .569   
676               Ivica Zubac   C  22  LAC  72  70  18.4  3.3   5.3  .613   

     ...  ORB  DRB   TRB  AST  STL  BLK  TOV   PF   PTS  YEAR  
0    ...  3

             Player   Pos   Age    Tm     G    GS    MP    FG   FGA   FG%  \
0        Quincy Acy    SF    23   TOT    63     0  13.4   1.0   2.2  .468   
1        Quincy Acy    SF    23   TOR     7     0   8.7   0.9   2.0  .429   
2        Quincy Acy    SF    23   SAC    56     0  14.0   1.1   2.3  .472   
3      Steven Adams     C    20   OKC    81    20  14.8   1.1   2.3  .503   
4       Jeff Adrien    PF    27   TOT    53    12  18.1   2.7   5.2  .520   
..              ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   
630      Nick Young    SG    28   LAL    64     9  28.3   6.0  13.9  .435   
631  Thaddeus Young    PF    25   PHI    79    78  34.4   7.4  16.2  .454   
632            None  None  None  None  None  None  None  None  None  None   
633     Cody Zeller     C    21   CHA    82     3  17.3   2.1   4.9  .426   
634    Tyler Zeller     C    24   CLE    70     9  15.0   2.2   4.1  .538   

     ...   ORB   DRB   TRB   AST   STL   BLK   TOV    PF   PTS  YEAR  
0   

In [19]:
#Remove duplicate rows for players who were traded (keep only TOT)
statsclean = statstotal.drop_duplicates(subset = ['Player', 'YEAR'], keep='first')

In [43]:
import unicodedata

#Replace accented characters with normalized alphabet
def remove_accents(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    return u"".join([c for c in nfkd_form if not unicodedata.combining(c)])

In [60]:
statsclean['Player'] = statsclean['Player'].astype('string')

<ipython-input-60-6ba7d2e3ef18>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  statsclean['Player'] = statsclean['Player'].astype('string')


In [62]:
statsclean['Player'] = statsclean['Player'].apply(remove_accents)

<ipython-input-62-1352e3b200a2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  statsclean['Player'] = statsclean['Player'].apply(remove_accents)
